In [38]:
# Code for GitHub

# Import Libraries

#Import Gradio
import gradio as gr

#Import helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os

#Import TensorFlow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [131]:
#Set up dataset constraints

#Define 6 categories
labels = ['plastic','metal','glass','trash','paper', 'cardboard']

path = r'C:/Users/Sammy/garbage/'

#Image parameters
WIDTH = 224
HEIGHT = 224

In [76]:
train_datagen = ImageDataGenerator(
        rescale=1./255)

train= train_datagen.flow_from_directory(
        path + 'Train/',
        target_size=(WIDTH, HEIGHT),
        batch_size=32,
        class_mode='categorical')

Found 1773 images belonging to 6 classes.


In [77]:
val_datagen = ImageDataGenerator(
        rescale=1./255)

val = val_datagen.flow_from_directory(
        path + 'Validation/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 252 images belonging to 6 classes.


In [42]:
# Import VGG16 ImageNet weights for initialisation

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(WIDTH, HEIGHT, 3))

conv_base.trainable = False

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [43]:
# Create the model

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 256)               6422784   
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 21,139,014
Trainable params: 6,424,326
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
# Train the Model

batch_size = 64
epochs = 60

history = model.fit(train, batch_size=batch_size, epochs=epochs,validation_data=val)

Epoch 1/60
56/56 [==============================] - 71s 1s/step - loss: 2.1716 - accuracy: 0.2916 - val_loss: 1.3906 - val_accuracy: 0.5079
Epoch 2/60
56/56 [==============================] - 70s 1s/step - loss: 1.5000 - accuracy: 0.3920 - val_loss: 1.4238 - val_accuracy: 0.3611
Epoch 3/60
56/56 [==============================] - 70s 1s/step - loss: 1.3910 - accuracy: 0.4174 - val_loss: 1.3145 - val_accuracy: 0.5913
Epoch 4/60
56/56 [==============================] - 69s 1s/step - loss: 1.3401 - accuracy: 0.4411 - val_loss: 1.3169 - val_accuracy: 0.4603
Epoch 5/60
56/56 [==============================] - 69s 1s/step - loss: 1.2958 - accuracy: 0.4597 - val_loss: 1.2997 - val_accuracy: 0.5833
Epoch 6/60
56/56 [==============================] - 70s 1s/step - loss: 1.2156 - accuracy: 0.4743 - val_loss: 1.2008 - val_accuracy: 0.5595
Epoch 7/60
56/56 [==============================] - 69s 1s/step - loss: 1.2081 - accuracy: 0.4800 - val_loss: 1.2709 - val_accuracy: 0.5794
Epoch 8/60
56/56 [==

In [132]:
# Define Gradio Image Prediction Function

def predict(img):
    img = img.reshape(-1, WIDTH, HEIGHT, 3)
    pred = model.predict(img).tolist()[0]
    return {labels[i]: pred[i] for i in range(6)}

In [133]:
# Define Gradio Constraints
image = gr.inputs.Image(shape=(WIDTH, HEIGHT))
label = gr.outputs.Label(num_top_classes=5)

In [134]:
# Add some example images for Gradio Interface

filenames_test = [path + 'Examples/paper/paper1.jpg', path + 'Examples/plastic/plastic1.jpg',
                 path + 'Examples/glass/glass1.jpg']

In [169]:
#Add background image

css_code="body {background-image: url('file=C:/Users/Sammy/garbage/background.jpg')}"

In [170]:
# Launch Gradio

gr.Interface(fn=predict, inputs=image, outputs=label, interpretation='default',
             examples = filenames_test, css=css_code).launch(debug='True', share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://29896.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 93ms/step
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x24ffa87f610>,
 'http://127.0.0.1:7860/',
 'https://29896.gradio.app')